In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime as dt
import requests
import numpy as np
import pandas as pd

In [5]:
artarr = np.zeros((210, 4), dtype='>U60')

In [15]:
def selection(driver, addrow, i, syllable):
    """Move into detailed info of each artists then crawl them"""
    driver.implicitly_wait(10)

    artistClick = driver.find_element_by_css_selector('#pageList > div.list_atist12.d_artist_list > ul > li:nth-child('+ str(i) +') > div > div > dl > dt > a')
    artistClick.click()

    driver.implicitly_wait(10)

    detailinfo = driver.find_element_by_css_selector('#conts > div.wrap_tab_atist > ul > li:nth-child(2) > a')
    detailinfo.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    artist = soup.select('#conts > div.wrap_dtl_atist > div > div.wrap_atist_info > p')
    features_dt = soup.select('#conts > div.section_atistinfo03 > dl > dt')
    features = soup.select('#conts > div.section_atistinfo03 > dl > dd')

    artistproc1 = artist[0].text.strip().replace("아티스트명","")
    artistproc2 = artistproc1.replace(u"\xa0", u" ")
    artarr[5*syllable+i+addrow, 0] = artistproc2

    j = 1
    if features_dt[0].text.strip() == "데뷔":
        if features_dt[2].text.strip() == "결성일":
            for k in [1, 3, 4]:
                feature = features[k].text.strip().split('\n')

                artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
                j = j + 1
        else:
            for k in range(1, 4):
                feature = features[k].text.strip().split('\n')
                artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
                j = j + 1
    elif features_dt[0].text.strip() == "활동년대":
         for k in range(0, 3):
            feature = features[k].text.strip().split('\n')
            artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
            j = j + 1

    driver.back()
    driver.back()

def korean(driver):
    """Crawl from ㄱ to ㅎ, Numbers and others"""
    for syllable in range(1, 17):
        for i in range(1, 6):
            click = driver.find_element_by_css_selector('#conts > div.wrap_finder.atist > dl > dd > div.wrap_btn_atist.first_child > button:nth-child('+str(syllable)+')')
            click.click()
            #((syllable-1)*5) + i-1
            selection(driver, -6, i, syllable)


def english(driver):
    """Crawl from A to Z"""
    for syllable in range(1, 27):
        for i in range(1, 6):
            click = driver.find_element_by_css_selector('#conts > div.wrap_finder.atist > dl > dd > div:nth-child(2) > button:nth-child('+str(syllable)+') > span > span')
            click.click()
            #((syllable-1)*5) + i-1 + 80
            selection(driver, 74, i, syllable)



def crawlArtistFeatures(url):
    """Crawl top 5 Artists' name, year, type and genre for each syllable"""
    res = requests.get(url, headers={"User-agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0 rv:11.0) like Gecko"})
    res.raise_for_status()
    driver = webdriver.Chrome()
    driver.get(url)
    
    korean(driver)
    english(driver)

    driver.quit()

crawlArtistFeatures('https://www.melon.com/artistplus/finder/index.htm')

In [16]:
artarr

array([['기리보이 (홍시영)', '2010 년대', '솔로|남성',
        '발라드, 댄스, 랩/힙합, R&B/Soul, 인디음악'],
       ['김필', '2010 년대', '솔로|남성',
        '발라드, 댄스, R&B/Soul, 인디음악, 록/메탈, 포크/블루스'],
       ['김동률', '1990, 2000, 2010 년대', '솔로|남성', '발라드'],
       ['김하온 (HAON)', '2010 년대', '솔로|남성', '발라드, 랩/힙합'],
       ['김소유 (김소희)', '2010 년대', '솔로|여성', '성인가요'],
       ['닐로(Nilo)', '2010 년대', '솔로|남성', '발라드'],
       ['나플라(nafla)', '2010 년대', '솔로|남성', '랩/힙합, 인디음악'],
       ['나얼 (유나얼)', '1990, 2000, 2010 년대', '솔로|남성',
        '발라드, R&B/Soul, CCM, 국내CCM'],
       ['뉴이스트', '2010 년대', '그룹|남성', '발라드, 댄스'],
       ['넬 (NELL)', '2000, 2010 년대', '그룹|남성',
        '발라드, 댄스, POP, 록/메탈, 일렉트로니카'],
       ['다비치', '2000, 2010 년대', '그룹|여성', '발라드, 댄스, 랩/힙합, R&B/Soul'],
       ['다이나믹 듀오', '2000, 2010 년대', '그룹|남성',
        '발라드, 댄스, 랩/힙합, R&B/Soul, 록/메탈, CCM, 국내CCM'],
       ['디오 (D.O.)', '2010 년대', '솔로|남성', '발라드, R&B/Soul'],
       ['더보이즈(THE BOYZ)', '2010 년대', '그룹|남성',
        '발라드, 댄스, R&B/Soul, 일렉트로니카'],
       ['드림캐쳐', '2010 년대', '그룹|여

In [17]:
df = pd.DataFrame(artarr, columns=['artists', 'year', 'type', 'genre'])

In [18]:
df.set_index('artists', inplace=True)

In [19]:
df['year'] = df['year'].str.replace(' 년대', '')
df['type'] = df['type'].str.replace('|', ', ')
df['type'] = df['type'].str.replace('솔로', 'Solo')
df['type'] = df['type'].str.replace('그룹', 'Group')
df['type'] = df['type'].str.replace('남성', 'Male')
df['type'] = df['type'].str.replace('여성', 'Female')
df['type'] = df['type'].str.replace('혼성', 'Both')
df['genre'] = df['genre'].str.replace('발라드', 'Ballad')
df['genre'] = df['genre'].str.replace('댄스', 'Dance')
df['genre'] = df['genre'].str.replace('랩/힙합', 'Rap/Hiphop')
df['genre'] = df['genre'].str.replace('인디음악', 'Indie')
df['genre'] = df['genre'].str.replace('록/메탈', 'Rock/Metal')
df['genre'] = df['genre'].str.replace('포크/블루스', 'Folk/Blues')
df['genre'] = df['genre'].str.replace('국내CCM', 'Korean CCM')
df['genre'] = df['genre'].str.replace('뉴에이지', 'Newage')
df['genre'] = df['genre'].str.replace('일렉트로니카', 'Electronica')
df['genre'] = df['genre'].str.replace('포크', 'Folk')
df['genre'] = df['genre'].str.replace('재즈', 'Jazz')
df['genre'] = df['genre'].str.replace('성인가요', 'Trot')
df['genre'] = df['genre'].str.replace('\(.*\)', '', regex = True)
df

,year,type,genre
artists,,,
기리보이 (홍시영),2010,"Solo, Male","Ballad, Dance, Rap/Hiphop, R&B/Soul, Indie"
김필,2010,"Solo, Male","Ballad, Dance, R&B/Soul, Indie, Rock/Metal, Fo..."
김동률,"1990, 2000, 2010","Solo, Male",Ballad
김하온 (HAON),2010,"Solo, Male","Ballad, Rap/Hiphop"
김소유 (김소희),2010,"Solo, Female",Trot
닐로(Nilo),2010,"Solo, Male",Ballad
나플라(nafla),2010,"Solo, Male","Rap/Hiphop, Indie"
나얼 (유나얼),"1990, 2000, 2010","Solo, Male","Ballad, R&B/Soul, CCM, Korean CCM"
뉴이스트,2010,"Group, Male","Ballad, Dance"


In [20]:
df['year'] = df['year'].map(lambda x: x.split(','))
df['type'] = df['type'].map(lambda x: x.split(','))
df['genre'] = df['genre'].map(lambda x: x.split(','))
df.columns

Index(['year', 'type', 'genre'], dtype='object')

In [21]:
# for index, row in df.iterrows():
#     print(row['year'])

In [24]:
df['bagOfWords'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        words = words + ' '.join(row[col])+ ' '
    row['bagOfWords'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bagOfWords'], inplace = True)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bagOfWords'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:5]

0    기리보이 (홍시영)
1            김필
2           김동률
3    김하온 (HAON)
4     김소유 (김소희)
Name: artists, dtype: object

In [26]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.70352647, 0.54433105, ..., 0.60302269, 0.4472136 ,
        0.37796447],
       [0.70352647, 1.        , 0.49236596, ..., 0.36363636, 0.40451992,
        0.34188173],
       [0.54433105, 0.49236596, 1.        , ..., 0.49236596, 0.54772256,
        0.3086067 ],
       ...,
       [0.60302269, 0.36363636, 0.49236596, ..., 1.        , 0.67419986,
        0.68376346],
       [0.4472136 , 0.40451992, 0.54772256, ..., 0.67419986, 1.        ,
        0.6761234 ],
       [0.37796447, 0.34188173, 0.3086067 , ..., 0.68376346, 0.6761234 ,
        1.        ]])

In [27]:
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [28]:
recommendations('기리보이 (홍시영)')

['40',
 'Crush (신효섭)',
 'GRAY (그레이)',
 'BewhY (비와이)',
 'Ugly Duck (선주경)',
 '매드클라운',
 '로꼬 (권혁우)',
 '헤이즈 (Heize)',
 'VIINI (권현빈)',
 '한요한']